In [ ]:
import os
os.environ["HF_HOME"] = r"./.cache"
import pathlib
import re
import json

from datasets import Dataset
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast, \
    Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq, \
    EncoderDecoderModel, AutoTokenizer
from tokenizers import processors
from peft import PeftModel
# from utils.metric import SacreBleu
from utils.dataset import Flores, WMTvat, EnJaDatasetMaker

In [ ]:
def tokenize_dataset(data, src_lang, type, encoder_tokenizer=None, decoder_tokenizer=None):
    trg_lang = "ja" if src_lang == "en" else "en"

    if type == "mBART":
        data = data.map(
            EnJaDatasetMaker._get_map_compute_mBART_tokenization(
                tokenizer=encoder_tokenizer
            )
        )
    elif type.startswith("BERT-GPT2"):
        data = data.map(
            EnJaDatasetMaker._get_map_compute_BERT_GPT2_tokenization(
                encoder_tokenizer=encoder_tokenizer, decoder_tokenizer=decoder_tokenizer
            )
        )
    else: raise ValueError()
    return data

def get_tokenizer(type, src_lang):
    trg_lang = "ja" if src_lang == "en" else "en"
    
    if type == "mBART":
        tok = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50", src_lang=f"{src_lang}_XX", tgt_lang=f"{trg_lang}_XX")
        return {
            "encoder_tokenizer": tok,
            "decoder_tokenizer": tok            
        }
    elif type.startswith("BERT-GPT2"):
        if src_lang == "en":
            encoder = "bert-base-uncased"
            decoder = "rinna/japanese-gpt2-small"
        else: # src_lang == "ja"
            encoder = "cl-tohoku/bert-base-japanese-v3"
            decoder = "gpt2"
        tok = {
            "encoder_tokenizer": AutoTokenizer.from_pretrained(encoder, use_fast=True),
            "decoder_tokenizer": AutoTokenizer.from_pretrained(decoder, use_fast=True)
        }
        if tok["decoder_tokenizer"].pad_token_id is None:
            tok["decoder_tokenizer"].pad_token_id = tok["decoder_tokenizer"].eos_token_id
        tok["decoder_tokenizer"]._tokenizer.post_processor = processors.TemplateProcessing(
            single="$A " + tok["decoder_tokenizer"].eos_token,
            special_tokens=[(tok["decoder_tokenizer"].eos_token, tok["decoder_tokenizer"].eos_token_id)],
        )
        return tok
    else: raise ValueError()

def get_base_model(type, src_lang):
    trg_lang = "ja" if src_lang == "en" else "en"
    
    if type == "mBART":
        model =  MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")
    elif type == "BERT-GPT2-xattn-LoRA":
        path_to_ckp = f"./.ckp/{src_lang}-{trg_lang}-BERT-GPT2-xattn/checkpoint-{25000}"
        model = EncoderDecoderModel.from_pretrained(path_to_ckp, local_files_only=True)
    elif type == "BERT-GPT2-xattn":
        model = None
    else: raise ValueError()
    return model

In [ ]:
def compute_ckp_gens(model, path_to_ckp, dataset, tokenizer):
    if model is not None:
        model = PeftModel.from_pretrained(model=model, model_id=path_to_ckp)
    else:
        model = EncoderDecoderModel.from_pretrained(path_to_ckp, local_files_only=True)
    
    # adding metrics requires to pass trg_lang
    # metrics = SacreBleu.get_mBART_metric(tokenizer=tokenizer, target_language=trg_lang)
    data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

    train_args = Seq2SeqTrainingArguments(
        report_to="none",
        prediction_loss_only=False,
        predict_with_generate=True,
        bf16=True,
        group_by_length=True,
        output_dir="./.ckp",
        length_column_name="length",
        label_smoothing_factor=0.2,
        per_device_eval_batch_size=8
    )
    gen_config = {
        "max_length" : 256,
        "early_stopping" : True,
        "no_repeat_ngram_size" : 4,
        "length_penalty" : 1.0,
        "num_beams" : 5
    }

    trainer = Seq2SeqTrainer(
        model,
        args=train_args,
        data_collator=data_collator,
        # compute_metrics=metrics
    )

    model.cuda()
    model.eval()
    predictions = trainer.predict(dataset, **gen_config).predictions
    predictions_decode = tokenizer["decoder_tokenizer"].batch_decode(predictions, skip_special_tokens=True)
        
    return predictions_decode
    

In [ ]:
def compute_generations(type, ckp_name, ckp_nums, dataset, src_lang):
    assert type in ["mBART", "BERT-GPT2-xattn", "BERT-GPT2-xattn-LoRA"], "invalid type"
    assert os.path.exists(f"./.ckp/{ckp_name}"), "invalid ckp id"
    assert src_lang in ["en", "ja"], "invalid language"
    trg_lang = "ja" if src_lang == "en" else "en"
    
    # get all checkpoints
    ckps = []
    p_num = re.compile(".*-(.*)$")
    for fname in pathlib.Path(f"./.ckp/{ckp_name}").glob("*"):
        ckps.append(int(p_num.match(str(fname)).groups()[0]))
    ckps.sort()
    assert all(num in ckps for num in ckp_nums), "ckp_nums is invalid"
    
    # generate tokenizer, dataset, model
    tokenizers = get_tokenizer(type, src_lang)
    model = get_base_model(type, src_lang=src_lang)
    processed = tokenize_dataset(dataset, src_lang, type, **tokenizers)
    
    # generate predictions with given model
    gens = {}
    for ckp in ckp_nums:
        path_to_ckp = f"./.ckp/{ckp_name}/checkpoint-{ckp}"
        gen = compute_ckp_gens(model, path_to_ckp, processed, tokenizers["decoder_tokenizer"])
        gens[f"pred@{ckp}"] = gen
    
    # create new dataset with source, target and predictions
    for key in dataset:
        gens[key] = dataset[key]
    data = Dataset.from_dict(gens)
    
    return data

### Generation (en, ja)

In [27]:
dataset = {
    "source": [
        "The number of users of the Yahoo! and Microsoft services combined will rival the number of AOL's customers.",
        "The game publisher Konami stated today in a Japanese newspaper that they will not be releasing the game Six Days in Fallujah.",
        "Present-day parts of Belgium were part of Luxembourg in the past but became Belgian after the 1830s Belgian Revolution.",
        
        "many animals have been destroyed by men .",
        "it saved me .",
        "i don 't blame you .",
        
        "There were two to three appointed to the post.",
        "In the last 3 months, over 80 arrestees were released from the Central Booking facility without being formally charged.",
        "He joined them in 1945 and stayed until 1958.",
        
        "An additional 300 brings the total to 1,300 carriages to be acquired to relieve overcrowding.",
        "There are ten dogs and five cats in this house.",
        "There are three dolphins.",
        
        "Today, I picked up a 10,000-yen bill.",
        "There are three minors aged 18."
    ],
    "target": [
        "ヤフーとマイクロソフトのサービスを合わせたユーザー数は、AOLの顧客数に匹敵するだろう。",
        "ゲームメーカーのコナミは本日、日本の新聞で、「Six Days in Fallujah」というゲームをリリースしないことを明言しました。",
        "現在のベルギー領の一部は過去にルクセンブルク領でしたが、1830年代のベルギー革命後にベルギー領になりました。",
        
        "多くの動物が人間によって滅ぼされた。",
        "あなたの存在に助けられたよ。",
        "あなたがそうするのは当然だ。",
        
        "定員は2～3人。",
        "過去3カ月間に、80人以上の逮捕者が正式に起訴されることなくセントラルブッキング施設から釈放されました。",
        "1945年に彼らと合流し、1958年まで滞在した。",
        
        "混雑を緩和するために、300両を追加して計1,300両が確保される予定です。",
        "この家には犬が10匹、猫が5匹います。",
        "イルカが3頭います。",
        
        "きょう、1万円札を拾いました。",
        "18歳の未成年が3人います。"
    ]
}

ja_additions = {
    "source": [
        "このエレベーターは一度に１０人運べる。", 
        "このエレベーターは１０人運ぶことができる。",
        "やることはいくらでもある。",
        "やることは山ほどある。",
        "やることがたくさんある。",
        "こちらは娘です。",
        "この子は私の娘です。",
        "これは娘です。",
        "当地に来てからどのくらいになりますか。",
        "ここに来てどのくらい？",
        "ここに来て、どれくらいになるの？",
        "あなたはどれぐらいの時間ここにいるのですか。",
        "すみません、田中先生はいますか。",
        "すみません、田中先生はいらっしゃいますか",
        "私はさくらです。",
        "私はさくらと申します。",
        "ぺらぺら", 
        "どきどき", 
        "ぺこぺこ"
    ],
    "target": [None] * 19
}

en_additions = {
    "source": [
        "This elevator is capable of carrying 10 persons at a time.",
        "I have a ton of stuff to do.",
        "This is my daughter.",
        "How long have you been here?",
        "Is Mr. Tanaka here?",
        "I am Sakura."
    ],
    "target": [None] * 6
}

In [ ]:
data = {
    "source": dataset["source"].copy(),
    "target": dataset["target"].copy(),
}
data["source"].extend(en_additions["source"])
data["target"].extend(en_additions["target"])
assert len(data["source"]) == len(data["target"])

gen_base      = compute_generations("mBART", "en-ja-mixed-250-250k+bt-250k", [2500, 25000]       , dataset, "en")

gen_extended  = compute_generations("mBART", "en-ja-mixed-500k"            , [2500, 45000, 50000], dataset, "en")

gen_baseandbt = compute_generations("mBART", "en-ja-ckp-25000-bt-500k"     , [2500, 35000, 50000], dataset, "en")

### Generation (ja, en)

In [ ]:
data = {
    "source": dataset["source"].copy(),
    "target": dataset["target"].copy(),
}
data["source"].extend(ja_additions["source"])
data["target"].extend(ja_additions["target"])
assert len(data["source"]) == len(data["target"])

gen_base      = compute_generations("mBART", "ja-en-mixed-250-250k+bt-250k", [2500, 25000]       , dataset, "ja")

gen_extended  = compute_generations("mBART", "ja-en-mixed-500k"            , [2500, 45000, 50000], dataset, "ja")

gen_baseandbt = compute_generations("mBART", "ja-en-ckp-25000-bt-500k"     , [2500, 35000, 50000], dataset, "ja")